In [52]:
import pandas as pd
import json
from web3 import Web3
from datetime import datetime, timedelta
import time
import sys
import json
from collections import Counter

# Map token names to contract addresses
token_address_map = {
    'rETH': '0xae78736Cd615f374D3085123A210448E74Fc6393'
    # fill with rest
}

# Map market name to "Pool" contract address and abi filepath
contract_address_abi_map = {
    'AAVE': ('0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2', './AAVE_Pool_ABI.json'),
    'COMPOUND': ('0xA17581A9E3356d9A858b789D68B4d866e593aE94', './Compound_ABI.json'),
    'PRISMA': (['0x0d6741f1A3A538F78009ca2e3a13F9cB1478B2d0', './Prisma_TroveManagerNew_ABI.json'], 
              ['0xe0e255FD5281bEc3bB8fa1569a20097D9064E445', './Prisma_TroveManagerOld_ABI.json'])
    # fill with rest
}

# Connect to ETH blockchain with infura API key
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/7e4f5238262543919688a59d0fef4a1d'))

In [53]:
# Find block number closest to timestamp with binary search
def find_block_by_timestamp(target_timestamp):
    low, high = 0, w3.eth.block_number
    while low < high:
        mid = (low + high) // 2
        mid_block_timestamp = w3.eth.get_block(mid).timestamp
        if mid_block_timestamp < target_timestamp:
            low = mid+1
        else:
            high = mid
    return low


# Helper to convert block number to readable timestamp string
def get_block_timestamp(block_num):
    block = w3.eth.get_block(block_num)
    timestamp = block.timestamp
    
    return timestamp


# Lists transactions of supplying rETH to specified markets
def supply_transactions(market, timeframe = 30, token = 'rETH'):
    """
    Returns all "Supply" transactions for rETH from the specified markets over a given timeframe expressed as days.
        (Example: supply_transactions(AAVE, 90, rETH) returns all supply transactions for rETH on the AAVE market in the past 90 days
        
    Parameters:
    - markets (str): The lending and borrowing market from which to fetch data. Defaults to 'AAVE'.
    - timeframe (int): The number of days back from the current date to fetch data. Defaults to 90 days.
    - token (str): The token we want to see supply transactions for. Defaults to rETH
    
    Returns:
    pandas.DataFrame: A DataFrame where each row corresponds to a unique supply transaction with the following columns:
        - 'User (Wallet Address)': The address for the user who initiated the transaction
        - 'Amount (Wei)': The amount supplied expressed in Wei
        - 'Amount (rETH)': The amount supplied expressed in rETH
        - 'Timestamp': The timestamp in 'YYYY-MM-DD HH:MM:SS' format.
        - 'LogIndex': The log index
        - 'TransactionIndex': The transaction index
        - 'TransactionHash': The hash for the transaction
        - 'BlockHash': The hash for the block
        - 'BlockNumber': The block number
    
    """
    supply_list = []
    # Lookup token address
    token_address = token_address_map[token]
    
    # Get timestamp of timeframe provided
    current_time = datetime.now()
    start_time = current_time - timedelta(days=timeframe)
    st_timestamp = int(start_time.timestamp())
    
    # Find the block associated with our start_time timestamp
    st_block = find_block_by_timestamp(st_timestamp)
    
    # Make sure the requested market is supported
    if market in contract_address_abi_map.keys():
        # Set up contract address and ABI for each market
        contract_address, abi_filepath = contract_address_abi_map[market]
        
        if (market != 'PRISMA'):
            # Web3 setup
            with open(abi_filepath) as f:
                abi = json.load(f)
            
            smart_contract = w3.eth.contract(address=contract_address, abi=abi)
        else:
            smart_contract = []
            with open(contract_address[1]) as f:
                abi = json.load(f)
            smart_contract.append(w3.eth.contract(address=contract_address[0], abi=abi))
            with open(abi_filepath[1]) as f:
                abi = json.load(f)
            smart_contract.append(w3.eth.contract(address=abi_filepath[0], abi=abi))

        # Here we will need some sort of if/else or switching logic to process unique calls for each market
        # In the meantime, we simply check for the only supported market, 'AAVE'
        if market == 'AAVE':
            supplies = smart_contract.events.Supply().get_logs(fromBlock=st_block,
                                                       toBlock='latest',
                                                       argument_filters={'reserve': token_address})
            # Consider simplifying supply_list.. don't know what information future markets will return, 
            # ... Will most likely trim this down to: Reserve, User, Amount (in token, not Wei), Timestamp, and block number
            supply_list = [
                {
                'Reserve': event['args']['reserve'],
                'User (Wallet Address)': event['args']['user'],
                'Amount (Wei)': event['args']['amount'],
                'Amount (rETH)': event['args']['amount']/10**18,
                'Timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                'LogIndex': event['logIndex'],
                'TransactionIndex': event['transactionIndex'],
                'TransactionHash': event['transactionHash'].hex(),
                'BlockHash': event['blockHash'].hex(),
                'BlockNumber': event['blockNumber']
                }
                for event in supplies
            ]
        elif market == 'COMPOUND':
            supplies = smart_contract.events.SupplyCollateral().get_logs(fromBlock=st_block,
                                                toBlock='latest',
                                                argument_filters={'asset': '0xae78736Cd615f374D3085123A210448E74Fc6393'})
            supply_list = [
                {
                'Reserve': event['args']['asset'],
                'User (Wallet Address)': event['args']['from'],
                'Amount (Wei)': event['args']['amount'],
                'Amount (rETH)': event['args']['amount']/10**18,
                'Timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                'LogIndex': event['logIndex'],
                'TransactionIndex': event['transactionIndex'],
                'TransactionHash': event['transactionHash'].hex(),
                'BlockHash': event['blockHash'].hex(),
                'BlockNumber': event['blockNumber']
                }
                for event in supplies
            ]
        elif market == 'PRISMA':
            all_supplies = []
            for contract in smart_contract:
                supplies = contract.events.CollateralSent().get_logs(fromBlock=st_block,
                                                       toBlock='latest')
                supply_list = [
                    {
                    'Reserve': '0xae78736Cd615f374D3085123A210448E74Fc6393',
                    'User (Wallet Address)': event['args']['_to'],
                    'Amount (Wei)': event['args']['_amount'],
                    'Amount (rETH)': event['args']['_amount']/10**18,
                    'Timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                    'LogIndex': event['logIndex'],
                    'TransactionIndex': event['transactionIndex'],
                    'TransactionHash': event['transactionHash'].hex(),
                    'BlockHash': event['blockHash'].hex(),
                    'BlockNumber': event['blockNumber']
                    }
                    for event in supplies
                ]
                all_supplies.extend(supply_list)
                
            supply_list = all_supplies
        else:
            print("UNSUPPORTED MARKET ENTERED")
            return
        
        # Push supply into dataframe
        recent_supplies = pd.DataFrame(supply_list)
        #recent_supplies['Timestamp'] = pd.to_datetime(recent_supplies[0]['Timestamp'])
        return recent_supplies
        
    # The market wasn't in our dictionary of supported markets
    else:
        print("UNSUPPORTED MARKET ENTERED")
        return []

"""
The above and below functions are extremely similar, the only difference being the API call they use, 
One function could have been used with some flow control and an additional parameter (User passes in "Borrow" or "Supply"
and the program runs accordingly); however, without knowing how future markets will behave, what their API calls will
exactly look like, and what they will return, it was decided to keep the two separate in case more complex logic is 
needed in handling either case specifically. 
"""


# Lists transactions of borrowing rETH to specified markets
def borrow_transactions(market, timeframe = 30, token = 'rETH'):
    """
    Returns all "Borrow" transactions for rETH from the specified markets over a given timeframe expressed as days.
        (Example: borrow_transactions(AAVE, 90, rETH) returns all borrow transactions for rETH on the AAVE market in the past 90 days
        
    Parameters:
    - markets (str): The lending and borrowing market from which to fetch data. Defaults to 'AAVE'.
    - timeframe (int): The number of days back from the current date to fetch data. Defaults to 90 days.
    - token (str): The token we want to see supply transactions for. Defaults to rETH
    
    Returns:
    pandas.DataFrame: A DataFrame where each row corresponds to a unique borrow transaction with the following columns:
        - 'User (Wallet Address)': The address for the user who initiated the transaction
        - 'Amount (Wei)': The amount supplied expressed in Wei
        - 'Amount (rETH)': The amount supplied expressed in rETH
        - 'Timestamp': The timestamp in 'YYYY-MM-DD HH:MM:SS' format.
        - 'LogIndex': The log index
        - 'TransactionIndex': The transaction index
        - 'TransactionHash': The hash for the transaction
        - 'BlockHash': The hash for the block
        - 'BlockNumber': The block number
    
    """
    # List for borrow_lists of all markets
    all_market_borrow_list = []
    
    # Lookup token address
    token_address = token_address_map[token]
    
    # Get timestamp of timeframe provided
    current_time = datetime.now()
    start_time = current_time - timedelta(days=timeframe)
    st_timestamp = int(start_time.timestamp())
    
    # Find the block associated with our start_time timestamp
    st_block = find_block_by_timestamp(st_timestamp)
    
    # Make sure the requested market is supported
    if market in contract_address_abi_map.keys():
        # Set up contract address and ABI for each market
        contract_address, abi_filepath = contract_address_abi_map[market]

        # Web3 setup
        with open(abi_filepath) as f:
            abi = json.load(f)

        smart_contract = w3.eth.contract(address=contract_address, abi=abi)

        # Here we will need some sort of if/else or switching logic to process unique calls for each market
        # In the meantime, we simply check for the only supported market, 'AAVE'
        if market == 'AAVE':
            borrows = smart_contract.events.Borrow().get_logs(fromBlock=st_block,
                                                       toBlock='latest',
                                                       argument_filters={'reserve': token_address})
            # Consider simplifying borrow_list.. don't know what information future markets will return, 
            # ... Will most likely trim this down to: Reserve, User, Amount (in token, not Wei), Timestamp, and block number
            borrow_list = [
                {
                    'Reserve': event['args']['reserve'],
                    'User (Wallet Address)': event['args']['user'],
                    'Amount (Wei)': event['args']['amount'],
                    'Amount (rETH)': event['args']['amount']/10**18,
                    'Timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
                    'LogIndex': event['logIndex'],
                    'TransactionIndex': event['transactionIndex'],
                    'TransactionHash': event['transactionHash'].hex(),
                    'BlockHash': event['blockHash'].hex(),
                    'BlockNumber': event['blockNumber']
                }
                for event in borrows
            ]
        elif market == 'COMPOUND':
            print("Compound currently doesn't support borrowing of rETH")
            return
        elif market == 'PRISMA':
            print("Prisma currently doesn't support borrowing of rETH")
            return
        else:
            print("FUTURE MARKETS HERE")
            return
        
        # Push borrows into dataframe
        recent_borrows = pd.DataFrame(borrow_list)
        #recent_supplies['Timestamp'] = pd.to_datetime(recent_supplies[0]['Timestamp'])
        return recent_borrows
    # The market wasn't in our dictionary of supported markets
    else:
        print("UNSUPPORTED MARKET ENTERED")
        return []
    

In [54]:
recent = supply_transactions('PRISMA')
recent

['0x0d6741f1A3A538F78009ca2e3a13F9cB1478B2d0', './Prisma_TroveManagerNew_ABI.json'] ['0xe0e255FD5281bEc3bB8fa1569a20097D9064E445', './Prisma_TroveManagerOld_ABI.json']


,Reserve,User (Wallet Address),Amount (Wei),Amount (rETH),Timestamp,LogIndex,TransactionIndex,TransactionHash,BlockHash,BlockNumber
0,0xae78736Cd615f374D3085123A210448E74Fc6393,0x731bA6d73718E86d6B12Fd373d648862de3f4056,2421000000000000000,2.421000,1711267139,245,117,0x4fb9751bd643b12ea246d0405e76c848994b27128a9f...,0x0ed58c0d3437c66943e7b6e093c693ceababd7670f9a...,19503077
1,0xae78736Cd615f374D3085123A210448E74Fc6393,0x192820CE84FA9eb457Fb228c386fE0ed22F7E33C,150000000000000000000,150.000000,1711627067,37,5,0xc248277fd2b27c1784d789272687c05064cd56bb6e50...,0xf1a41037623872098ffaed8bc9d1f3f82e906a794bae...,19532459
2,0xae78736Cd615f374D3085123A210448E74Fc6393,0x3B15CEc2d922Ab0Ef74688bcC1056461049f89CB,11078216393329209654,11.078216,1711628939,223,116,0x6710b3c1d8bd975c2d38e969ef3c405b6d99ca52c8af...,0x436d11221d185e4a77d01a3848f99355f1bbe708e1b1...,19532612
3,0xae78736Cd615f374D3085123A210448E74Fc6393,0x1fEC6F448Eb3ff479Ea9682F7eFf21A69762318f,34000000000000000000,34.000000,1711636655,128,57,0x1624fda3865b9f142b3c0d1b705d4fec422669149c61...,0xe777fd4cbcf7603a8d8f7232c31d211f333dae168b08...,19533235
4,0xae78736Cd615f374D3085123A210448E74Fc6393,0x51A125F89CBE0bFBFD62FeE8a750AAeC8fabC441,10497966470149292860,10.497966,1711640171,220,186,0xfef2d5849018f0e5c7eb1e126d9eaa7ea3f696051c97...,0xab396af9df4b8b2d1fc91eff7e72687963899f5944db...,19533517
...,...,...,...,...,...,...,...,...,...,...
109,0xae78736Cd615f374D3085123A210448E74Fc6393,0x5c15ceD59fD964fCFFF2d2F376b4a40b6D1Fce1c,2290936307831773000,2.290936,1712501903,173,90,0x36847665521a94fb88caad9ff99b5a0d7a18d333cb19...,0x04f0421d0dbdc376cee5fa766b4afef9a36bc419dfa3...,19604559
110,0xae78736Cd615f374D3085123A210448E74Fc6393,0xEaa44e71a7723867e20fcc0a17045a5CfF621B8E,4549510035699037054,4.549510,1712604611,364,273,0xa5f61cf9507f5c72a89d5397391dcdb38ef5d9291cd0...,0xcd76a013779c1cd555d78d0cec0628ac11079c77af7f...,19613039
111,0xae78736Cd615f374D3085123A210448E74Fc6393,0x5c15ceD59fD964fCFFF2d2F376b4a40b6D1Fce1c,2741576764724075646,2.741577,1712651735,114,64,0x26554f7623bccc8f3c0728f83d5546579b96c6fefb54...,0x3cf845327f006a1685dd5162d7daed1b2c3726c0ded8...,19616933
112,0xae78736Cd615f374D3085123A210448E74Fc6393,0x5c15ceD59fD964fCFFF2d2F376b4a40b6D1Fce1c,4000000000000000000,4.000000,1712716259,143,106,0x5f79409a0bde7ce8ccd4d4aa24fe6e7c33073079dd77...,0x61c69620cc9af3cc7a303469941aba1617fb8405e83b...,19622263


In [44]:
with open('./Prisma_TroveManagerOld_ABI.json') as f:
            abi = json.load(f)
smart_contract = w3.eth.contract(address='0xe0e255FD5281bEc3bB8fa1569a20097D9064E445', abi=abi)

supplies = smart_contract.events.CollateralSent().get_logs(fromBlock='earliest',
                                                       toBlock='latest')
supplies

(AttributeDict({'args': AttributeDict({'_to': '0xDb92A7a37A9e7D0a3Cab61e22DFf62FfEDfC2E4f',
   '_amount': 1549971107790175748}),
  'event': 'CollateralSent',
  'logIndex': 90,
  'transactionIndex': 11,
  'transactionHash': HexBytes('0xf896f760ac27602d90a27d63ebf26b28ed4ec394c2540e7262ed04a9657f6318'),
  'address': '0xe0e255FD5281bEc3bB8fa1569a20097D9064E445',
  'blockHash': HexBytes('0x8c331182dedf7aebaedb1ca3bef4a9d8894fd2427e2e359beb5a633a8572e0f4'),
  'blockNumber': 18043186}),
 AttributeDict({'args': AttributeDict({'_to': '0xDb92A7a37A9e7D0a3Cab61e22DFf62FfEDfC2E4f',
   '_amount': 1691767544280099852}),
  'event': 'CollateralSent',
  'logIndex': 73,
  'transactionIndex': 14,
  'transactionHash': HexBytes('0x4ba56dd1a0466e1255a6886ded9e47b07fe936cd11c03961ffc5bd72649aac99'),
  'address': '0xe0e255FD5281bEc3bB8fa1569a20097D9064E445',
  'blockHash': HexBytes('0x9ef57dc3288889d866cecf3aea9238c1a145632fc28d50f9c35cd3660aad414c'),
  'blockNumber': 18043263}),
 AttributeDict({'args': Attr

In [37]:
contract_address, abi_filepath = contract_address_abi_map['PRISMA']
print(contract_address[0], contract_address[1])
print(abi_filepath[0], abi_filepath[1])

with open (abi_filepath[1]) as f:
    abi = json.load(f)

0x0d6741f1A3A538F78009ca2e3a13F9cB1478B2d0 ./Prisma_TroveManagerNew_ABI.json
0xe0e255FD5281bEc3bB8fa1569a20097D9064E445 ./Prisma_TroveManagerOld_ABI.json
